# Description

 The goal of this notebook is to find the better doctors in Canada on a site called [RateMDs](https://www.ratemds.com), and avoid being victim of bad doctor practices which is unfortunately not uncommon. You are welcome to modify the code to suit your needs. The project is inspired by [this repository](https://github.com/rbanerjee3) of Rupam Anup Banerjee.

## How does it work?

This program has 2 components:
1. crawler
2. analyzer

The crawler is used to retrieve all the specialty doctors in a region from ratemds.com, so the analyzer can process filtering based on rating and disease keywords. The crawler will also save the retrieved doctor information to a csv file as it can take a lot of time to crawl (we respect the site's throughput and are intentionally slow).

The crawler registers the following information in RateMDs:
* Name of the doctor
* Specialty
* Star-rating
* No. of reviews
* Star-ratings of reviews
* Review contents

You need to provide the following information to the crawler:
* Doctor location
* Doctor specialty

The analyzer filters doctors based on rating and the comments by other patients. You need to provide the following information to the analyzer:
* Number of candidate doctors
* Disease keywords

The number of candidate doctors indicate how many candidate doctors will be retrieved based on rating, while disease keywords will help find doctors who have practised treatment of your specific disease. For example, an ophthalmologist can mainly do corneal vs retinal diseases.

You can also provide some additional information to the analyzer:
* Minimum number of reviews 
* Minimum overall rating threshold
* Number of top and bottom ratings to analyze 

You can also optionally ask ChatGPT to give a summary of the filtered doctors based on patient comments. This however requires a ChatGPT API key and you need to pay OpenAI for their API.

## Available locations and doctor specialties

In [1]:
avail_locations = {
    "montreal" : "qc/montreal/",
    "toronto" : "on/toronto/",
    "vancouver" : "bc/vancouver/"
}

avail_specialties = {
    "acupuncturist": "acupuncturist/",
    "addiction medicine specialist": "addiction-medicine/",
    "allergist / immunologist": "allergist-immunologist/",
    "allergist": "allergist-immunologist/",
    "immunologist": "allergist-immunologist/",
    "anesthesiologist": "anesthesiologist/",
    "audiologist": "audiologist/",
    "bariatric / weight loss specialist": "bariatric-weight-loss/",
    "bariatric": "bariatric-weight-loss/",
    "weight loss specialist": "bariatric-weight-loss/",
    "cardiologist": "cardiologist/",
    "cardiothoracic surgeon": "cardiothoracic-surgeon/",
    "chiropractor": "chiropractor/",
    "colorectal surgeon / proctologist": "colorectal-proctologist/",
    "colorectal surgeon": "colorectal-proctologist/",
    "proctologist": "colorectal-proctologist/",
    "dentist": "dentist/",
    "dermatologist": "dermatologist/",
    "dietitian": "dietitian/",
    "emergency room doctor": "emergency-critical/",
    "endocrinologist": "endocrinologist/",
    "endodontist": "endodontist",
    "family doctor / general practitioner": "family-gp/",
    "family doctor": "family-gp/",
    "general practitioner": "family-gp/",
    "gastroenterologist": "gastroenterologist/",
    "general surgeon": "surgeon-general/",
    "geneticist": "geneticist/",
    "gynecologists and obstetricians (obgyn)": "gynecologist-obgyn/",
    "gynecologist": "gynecologist-obgyn/",
    "obstetrician": "gynecologist-obgyn/",
    "obgyn": "gynecologist-obgyn/",
    "homeopath": "homeopath/",
    "infectious disease specialist": "infectious-disease/",
    "internist / geriatrician": "internist-geriatrician/",
    "internist": "internist-geriatrician/",
    "geriatrician": "internist-geriatrician/",
    "massage therapist": "registered-massage-therapist/",
    "midwife": "midwife/",
    "naturopath": "naturopath/",
    "nephrologist": "nephrologist/",
    "neurologist": "neurologist/",
    "neurosurgeon": "neurosurgeon/",
    "nurse practitioner": "nurse-practitioner/",
    "occupational therapist": "occupational-therapist/",
    "oncologist / hematologist": "oncologist-hematologist/",
    "oncologist": "oncologist-hematologist/",
    "hematologist": "oncologist-hematologist/",
    "ophthalmologist": "ophthalmologist/",
    "optometrist": "optometrist/",
    "oral surgeon": "oral-surgeon/",
    "orthodontist": "orthodontist/",
    "orthopedic surgeon": "orthopedics-sports/",
    "osteopath": "osteopath/",
    "otolaryngologist (ent)": "ear-nose-and-throat-ent/",
    "otolaryngologist": "ear-nose-and-throat-ent/",
    "ent": "ear-nose-and-throat-ent/",
    "pain management specialist / physical therapist": "pain-mgmt-physical-rehab/",
    "pain management specialist": "pain-mgmt-physical-rehab/",
    "pain medicine specialist": "pain-medicine-specialist/",
    "pathologist": "pathologist/",
    "pediatrician": "pediatrician/",
    "perinatologist / maternal-fetal medicine specialist": "perinatologist-maternal-fetal-medicine/",
    "perinatologist": "perinatologist-maternal-fetal-medicine/",
    "maternal-fetal medicine specialist": "perinatologist-maternal-fetal-medicine/",
    "periodontist": "periodontist/",
    "physiatrist / physical medicine & rehabilitation": "physiatrist-physical-medicine-rehabilitation/",
    "physiatrist": "physiatrist-physical-medicine-rehabilitation/",
    "physical medicine & rehabilitation": "physiatrist-physical-medicine-rehabilitation/",
    "physical therapist / physiotherapist": "physical-therapist-physiotherapist/",
    "physical therapist": "physical-therapist-physiotherapist/",
    "physiotherapist": "physical-therapist-physiotherapist/",
    "physician assistant": "physician-assistant/",
    "plastic / cosmetic surgeon, physician": "cosmetic-plastic-surgeon/",
    "plastic": "cosmetic-plastic-surgeon/",
    "cosmetic surgeon, physician": "cosmetic-plastic-surgeon/",
    "podiatrist": "podiatrist/",
    "psychiatrist": "psychiatrist/",
    "psychologist": "psychologist/",
    "pulmonologist": "pulmonologist/",
    "radiation oncologist": "radiation-oncologist/",
    "radiologist": "radiologist/",
    "reproductive endocrinologist": "reproductive-endocrinologist/",
    "rheumatologist": "rheumatologist/",
    "sleep doctor": "sleep-disorders/",
    "sports medicine physician": "sports-medicine-physicians/",
    "therapist": "therapist/",
    "urogynecologist": "urogynecologist/",
    "urologist": "urologist/",
    "vascular surgeon / phlebologist": "vascular-phlebologist/",
    "vascular surgeon": "vascular-phlebologist/",
    "phlebologist": "vascular-phlebologist/"
}

# Libraries

In [2]:
#Import Webscraping libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException

#Import Data structure libraries
import pandas as pd

#Other libraries
from time import sleep

# Parameters

In [3]:
# CRAWLER PARAMETERS
DOCTOR_LOCATION = "Montreal"
DOCTOR_SPECIALTY = "ophthalmologist"

# ANALYZER PARAMETERS
DISEASE_KEYWORDS = ["diabetic", "diabetes", "retinopathy", "macular degeneration", "retina", "diabétique", "diabète", "rétinopathie", "dégénérescence maculaire", "rétine"]
NUM_DOCTORS_TO_RETRIEVE = 50
NUM_TOP_AND_BOTTOM_REVIEWS_TO_ANALYZE = 3
MIN_NUM_RATINGS_THRESHOLD = 5
MIN_OVERALL_RATING_THRESHOLD = 4

# OPTIONAL PARAMETERS
OPENAI_API_KEY = ""

assert DOCTOR_LOCATION.lower() in avail_locations.keys(), "Please provide a valid location. By default, only Montreal, Toronto and Vancouver are available. Modify <avail_locations> above to add more."
assert DOCTOR_SPECIALTY.lower() in avail_specialties.keys(), "Please provide a valid specialty. Check the options above."

# SYSTEM PARAMETERS
IGNORED_EXCEPTIONS = (NoSuchElementException, StaleElementReferenceException)
WEBDRIVER_WAIT_TIME = 10
WEBDRIVER_MAX_RETRY = 5

## Crawler code

In [4]:
url = f"https://www.ratemds.com/best-doctors/{avail_locations[DOCTOR_LOCATION.lower()]}{avail_specialties[DOCTOR_SPECIALTY.lower()]}"

In [5]:
def create_doctor_dataframe():
    '''Create data structure for crawled information'''
    return pd.DataFrame(columns=[
        'Doctor name',
        'Specialty',
        'Overall rating',
        'No. of reviews',
        'Reviews',
        'Review rating'
    ])

def launch_chrome(url):
    '''Launch a Chrome Session'''
    driver=webdriver.Chrome()
    driver.implicitly_wait(WEBDRIVER_WAIT_TIME)
    driver.get(url)
    return driver

def _stable_find_page_doctors(page_doctor_list):
    '''Extract single doctor reviews per page'''
    page_doctors = []
    for i in range(WEBDRIVER_MAX_RETRY):
        try:
            for doctor in page_doctor_list:
                page_doctors.append((doctor.text, doctor.get_attribute('href')))
            break
        except StaleElementReferenceException:
            sleep(1)
            continue
    return page_doctors

def stable_find_doctor_list(driver):
    '''Find all doctor names and urls for the given region and specialty'''
    doctors = []
    while True:
        for i in range(WEBDRIVER_MAX_RETRY):
            try:
                page_doctor_list = WebDriverWait(driver, WEBDRIVER_WAIT_TIME, ignored_exceptions=IGNORED_EXCEPTIONS).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, 'search-item-doctor-name'))
                )
                page_doctors = _stable_find_page_doctors(page_doctor_list)
                doctors += page_doctors
                next_page_link = driver.find_element(By.XPATH, '//a[@aria-label="Next page"]').get_attribute('href')
                driver.get(next_page_link)
                sleep(1)
                break
            except StaleElementReferenceException:
                sleep(1)
                continue
            except NoSuchElementException:
                return doctors

def stable_find_doctor_info(driver, url):
    '''Extract single doctor general info'''
    driver.get(url)
    sleep(1)
    for i in range(WEBDRIVER_MAX_RETRY):
        try:
            doctor_banner_panel = WebDriverWait(driver, WEBDRIVER_WAIT_TIME, ignored_exceptions=IGNORED_EXCEPTIONS).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'doctor-banner-panel'))
            )
            specialty = WebDriverWait(doctor_banner_panel, WEBDRIVER_WAIT_TIME, ignored_exceptions=IGNORED_EXCEPTIONS).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'search-item-info'))
            ).text
            leave_review = WebDriverWait(doctor_banner_panel, WEBDRIVER_WAIT_TIME, ignored_exceptions=IGNORED_EXCEPTIONS).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'leave-a-review-cta-text'))
            ).text
            if leave_review == 'Be the First to Review!':
                return specialty, '', ''
            rating_and_no_reviews = WebDriverWait(doctor_banner_panel, WEBDRIVER_WAIT_TIME, ignored_exceptions=IGNORED_EXCEPTIONS).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'star-rating-count'))
            ).text
            overall_rating = rating_and_no_reviews[:3]
            no_reviews = rating_and_no_reviews[5:rating_and_no_reviews.find(' review')]
            break
        except StaleElementReferenceException:
            sleep(1)
            continue
        except TimeoutException:
            driver.refresh()
            sleep(1)
            continue

    return specialty, overall_rating, no_reviews

def _get_star_rating(driver):
    '''Extract review star rating'''
    rating = 0
    stars = WebDriverWait(driver, WEBDRIVER_WAIT_TIME, ignored_exceptions=IGNORED_EXCEPTIONS).until(
        EC.presence_of_all_elements_located((By.TAG_NAME, 'svg'))
    )
    for star in stars:
        if star.get_attribute('class') == 'selected': rating += 1
        elif star.get_attribute('class') == 'half': rating += 0.5
    return rating

def _stable_find_page_reviews(page_review_list):
    '''Extract single doctor reviews per page'''
    page_reviews = []
    for i in range(WEBDRIVER_MAX_RETRY):
        try:
            for review in page_review_list:
                star_rating = _get_star_rating(review)
                comment = WebDriverWait(review, WEBDRIVER_WAIT_TIME, ignored_exceptions=IGNORED_EXCEPTIONS).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'rating-comment-body'))
                ).text
                page_reviews.append((star_rating, comment))
            break
        except StaleElementReferenceException:
            sleep(1)
            continue
    return page_reviews

def stable_find_doctor_reviews(driver):
    '''Extract single doctor reviews'''
    reviews = []
    while True:
        for i in range(WEBDRIVER_MAX_RETRY):
            try:
                page_review_list = WebDriverWait(driver, WEBDRIVER_WAIT_TIME, ignored_exceptions=IGNORED_EXCEPTIONS).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, 'rating'))
                )
                page_reviews = _stable_find_page_reviews(page_review_list)
                reviews += page_reviews
                next_page_link = driver.find_element(By.XPATH, '//a[@aria-label="Next page"]').get_attribute('href')
                driver.get(next_page_link)
                sleep(1)
                break
            except StaleElementReferenceException:
                sleep(1)
                continue
            except NoSuchElementException:
                return reviews
            
def register_doctor_info_to_dataframe(dataframe, doctor_name, specialty, overall_rating, no_reviews, reviews):
    '''Register single doctor information to pandas dataframe'''
    for review in reviews:
        dataframe.loc[len(dataframe)] = {
            'Doctor name': doctor_name,
            'Specialty': specialty,
            'Overall rating': overall_rating,
            'No. of reviews': no_reviews,
            'Reviews': review[1],
            'Review rating': review[0]
        }


In [7]:
print('creating data frame for doctor info...')
doctors_data = create_doctor_dataframe()
driver = launch_chrome(url)
driver.maximize_window()
print(f'extracting list of {DOCTOR_SPECIALTY} in {DOCTOR_LOCATION}...')
doctors = stable_find_doctor_list(driver)
print(f'number of {DOCTOR_SPECIALTY} found: {len(doctors)}.')

for doctor_name, link in doctors:
    print(f'extracting information of {doctor_name}...')
    specialty, overall_rating, no_reviews = stable_find_doctor_info(driver, link)
    print(f'specialty: {specialty}, overall_rating: {overall_rating}, no_reviews: {no_reviews}')
    # no review
    if overall_rating == '' and no_reviews == '':
        no_reviews = 0
        reviews = [('', '')]
    # with review(s)
    else:
        reviews = stable_find_doctor_reviews(driver)
        try:
            assert len(reviews) == int(no_reviews), f"Crawler error: all reviews are not retrieved for {doctor_name}. Extraction omitted, please check manually."
        except AssertionError:
            continue
    print('registering information to dataframe...')
    register_doctor_info_to_dataframe(doctors_data, doctor_name, specialty, overall_rating, no_reviews, reviews)
print('saving data to csv file...')    
doctors_data.to_csv(f'{DOCTOR_SPECIALTY}_{DOCTOR_LOCATION}.csv')

creating data frame for doctor info...
extracting list of ophthalmologist in Montreal...
number of ophthalmologist found: 311.
extracting information of Dr. Marc Mullie...
specialty: Ophthalmologist, overall_rating: 4.9, no_reviews: 267
registering information to dataframe...
extracting information of Dr. Avi Wallerstein...
specialty: Ophthalmologist, overall_rating: 4.9, no_reviews: 224
registering information to dataframe...
extracting information of Dr. Mona Harrissi Dagher...
specialty: Ophthalmologist, overall_rating: 4.9, no_reviews: 61
registering information to dataframe...
extracting information of Dr. Evan Kalin-Hajdu...
specialty: Ophthalmologist, overall_rating: 4.9, no_reviews: 57
registering information to dataframe...
extracting information of Dr. Darren L. Albert...
specialty: Ophthalmologist, overall_rating: 4.8, no_reviews: 188
registering information to dataframe...
extracting information of Dr. Pierre Demers...
specialty: Ophthalmologist, overall_rating: 4.9, no_rev

# Analyzer Code

In [5]:
dtype_dict = {
    'Doctor name': 'str',
    'Specialty': 'str',
    'Overall rating': 'float',
    'No. of reviews': 'int',
    'Reviews': 'str',
    'Review rating': 'float'
}

doctors_data = pd.read_csv(f'{DOCTOR_SPECIALTY}_{DOCTOR_LOCATION}.csv', dtype=dtype_dict)

In [6]:
def get_doctors_with_N_or_more_reviews(dataframe, N):
    return dataframe[dataframe['No. of reviews'] >= N]

def get_doctors_with_X_or_higher_rating(dataframe, X):
    return dataframe[dataframe['Overall rating'] >= X]

def get_doctors_with_experience_in_disease(dataframe):
    pattern = '|'.join(DISEASE_KEYWORDS)
    reviews_mentioning_disease = dataframe['Reviews'].str.contains(pattern, case=False, na=False)
    doctors_with_experience_in_disease = dataframe.loc[reviews_mentioning_disease, 'Doctor name'].unique()
    return dataframe[dataframe['Doctor name'].isin(doctors_with_experience_in_disease)]

def group_by_doctor_names(dataframe):
    return dataframe.groupby('Doctor name').agg({
        'Overall rating': 'first',
        'Specialty': 'first',
        'No. of reviews': 'first',
        'Reviews': lambda x: list(x),
        'Review rating': lambda x: list(x)
    })

def get_N_highest_rated_reviews(dataframe, N):
    def get_top_reviews(row):
        reviews_with_ratings = list(zip(row['Reviews'], row['Review rating']))
        top_reviews = sorted(reviews_with_ratings, key=lambda x:x[1], reverse=True)[:N]
        return [(rating, review) for review, rating in top_reviews]
    dataframe[f'Top {N} reviews'] = dataframe.apply(get_top_reviews, axis=1)
    return dataframe

def get_N_lowest_rated_reviews(dataframe, N):
    def get_bottom_reviews(row):
        reviews_with_ratings = list(zip(row['Reviews'], row['Review rating']))
        bottom_reviews = sorted(reviews_with_ratings, key=lambda x:x[1])[:N]
        return [(rating, review) for review, rating in bottom_reviews]
    dataframe[f'Bottom {N} reviews'] = dataframe.apply(get_bottom_reviews, axis=1)
    return dataframe

def get_reviews_mentioning_disease(dataframe):
    def get_disease_reviews(row):
        reviews_with_ratings = list(zip(row['Reviews'], row['Review rating']))
        return [(rating, review) for review, rating in reviews_with_ratings if not pd.isna(review) and any(disease.lower() in review.lower() for disease in DISEASE_KEYWORDS)]
    dataframe['Reviews mentioning disease'] = dataframe.apply(get_disease_reviews, axis=1)
    dataframe['No. of reviews mentioning disease'] = dataframe['Reviews mentioning disease'].apply(len)
    dataframe['Percentage of reviews mentioning disease'] = dataframe.apply(lambda row: round(row['No. of reviews mentioning disease'] / len(row['Reviews']) * 100, 1), axis=1)
    return dataframe
    

In [16]:
def summarize_doctor_reviews_using_GPT(dataframe, API_key, N):
    if API_key == "": return dataframe
    import os
    import time
    import openai
    os.environ["OPENAI_API_KEY"] = API_key
    client = openai.OpenAI()
    def _query_llm(prompt, client, temperature=1, max_tokens=300):
        while True:
          try:
            completion = client.chat.completions.create(
              model='gpt-4o',
              messages=[
                {"role": "system", "content": "You are a helpful assistant that summarize patient reviews about doctors to help them make decisions about their illnesses."},
                {"role": "user", "content": prompt}
              ],
              n=1,
              temperature=temperature,
              max_tokens=max_tokens,
            )
            generations = [choice.message.content for choice in completion.choices]
            return generations
          except openai.RateLimitError as e:
            if 'Rate limit reached for requests' in str(e):
              print('OpenAI API request exceeded rate limit:\nRetrying in 60 sec...')
              time.sleep(60)
            else:
              print('You exceeded your current quota, please check your plan and billing details.')
              break
          except openai.APIError as e:
            print(f'OpenAI API error:\n{e}')
            break
          except Exception as e:
            print(f'OpenAI API call unsuccessful with following error:\n{e}')
            break

    def summarize(row):
        gpt_prompt = f"Please summarize the doctor based on patient reviews. The reviews are out of 5. Please summarize succinctly based on all the reviews and not individually. Up to {N} top reviews, {N} bottom reviews and all the reviews concerning a relevant disease are given below: \n"
        for top_review in row[f'Top {N} reviews']:
            gpt_prompt += f"top review: {top_review}\n"
        for bottom_review in row[f'Bottom {N} reviews']:
            gpt_prompt += f"bottom review: {bottom_review}\n"
        for review_mentioning_disease in row['Reviews mentioning disease']:
            gpt_prompt += f"reviews mentioning disease: {review_mentioning_disease}\n"
        gpt_prompt += "Please pay more attention to the reviews mentioning the relevant disease and the bottom reviews. In case of bottom reviews, please make a judgement on whether they are generalizable or exceptions.\n"
        gpt_prompt
    
        summary = _query_llm(gpt_prompt, client)
        return summary

    dataframe['GPT summary of doctor'] = dataframe.apply(summarize, axis=1)
    return dataframe

In [17]:
# first, get doctors with 5 or more reviews
filtered_doctors_data = get_doctors_with_N_or_more_reviews(doctors_data, MIN_NUM_RATINGS_THRESHOLD)
# second, get doctors with a rating higher than 4
filtered_doctors_data = get_doctors_with_X_or_higher_rating(filtered_doctors_data, MIN_OVERALL_RATING_THRESHOLD)
# third, get doctors who has experience treating a disease based on keywords
filtered_doctors_data = get_doctors_with_experience_in_disease(filtered_doctors_data)
filtered_doctors_data = group_by_doctor_names(filtered_doctors_data)
# fourth, get top 3 review, bottom 3 reviews and all reviews mentioning disease for each doctor
filtered_doctors_data = get_N_highest_rated_reviews(filtered_doctors_data, NUM_TOP_AND_BOTTOM_REVIEWS_TO_ANALYZE)
filtered_doctors_data = get_N_lowest_rated_reviews(filtered_doctors_data, NUM_TOP_AND_BOTTOM_REVIEWS_TO_ANALYZE)
filtered_doctors_data = get_reviews_mentioning_disease(filtered_doctors_data)
# (OPTIONAL) fifth, let chatGPT summarize a doctor based on top comments, bottom comments and comments mentioning your disease, requires openai API key
filtered_doctors_data = summarize_doctor_reviews_using_GPT(filtered_doctors_data, OPENAI_API_KEY, NUM_TOP_AND_BOTTOM_REVIEWS_TO_ANALYZE)
# last, rank doctors based on rating
top_doctors = filtered_doctors_data.sort_values('Overall rating', ascending=False).head(NUM_DOCTORS_TO_RETRIEVE)

In [18]:
top_doctors.to_csv(f'top_{DOCTOR_SPECIALTY}_in_{DOCTOR_LOCATION}.csv')
top_doctors

,Overall rating,Specialty,No. of reviews,Reviews,Review rating,Top 3 reviews,Bottom 3 reviews,Reviews mentioning disease,No. of reviews mentioning disease,Percentage of reviews mentioning disease,GPT summary of doctor
Doctor name,,,,,,,,,,,
Dr. Rachel Trussart,5.0,Ophthalmologist,19,"[Excellente médecin, toujours disponible et ag...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[(5.0, Excellente médecin, toujours disponible...","[(4.5, Met son patient en confiance, bonnes ex...","[(5.0, En pleine crise de la COVID, cette méde...",5,26.3,[Dr. Trussart appears to be an exceptional oph...
Dr. Amer Omar,4.9,Ophthalmologist,24,[Great dr. Take his time to explain exactly th...,"[5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[(5.0, Great dr. Take his time to explain exac...","[(3.0, C’est un spécialiste sûrement tres qual...","[(5.0, I was first sent to Dr. Omar for a tear...",3,12.5,[Dr. Omar appears to be a highly skilled and k...
Dr. Darren L. Albert,4.8,Ophthalmologist,188,[Merci Dr Albert ! Je suis très reconnaissante...,"[5.0, 4.5, 5.0, 4.5, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[(5.0, Merci Dr Albert ! Je suis très reconnai...","[(1.0, Quel docteur arrogant et prétentieux. P...","[(5.0, I had a big retina problem and Dr Alber...",3,1.6,[Dr. Darren Albert receives mixed reviews from...
Dr. Marc Andre Rheaume,4.8,Ophthalmologist,30,"[Docteur compétent et efficace, il a totalemen...","[5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 4.0, 5.0, ...","[(5.0, Docteur compétent et efficace, il a tot...","[(4.0, Superbe équipe et dr Rheaume est un spé...","[(5.0, Docteur compétent et efficace, il a tot...",4,13.3,[Dr. Rhéaume is highly regarded by his patient...
Dr. Shawn Cohen,4.7,Ophthalmologist,105,[Dr Cohen is by far the most knowledgeable and...,"[5.0, 5.0, 5.0, 5.0, 1.0, 5.0, 5.0, 5.0, 5.0, ...","[(5.0, Dr Cohen is by far the most knowledgeab...","[(1.0, my eyes were swollen ,he ignored my con...","[(5.0, What a Great Doctor.\n\nI initially wen...",2,1.9,[Dr. Cohen is highly praised for his knowledge...
Dr. Sébastien Olivier,4.7,Ophthalmologist,42,[Je suis arrivé à l'urgence de l'hôpital Maiso...,"[4.5, 5.0, 4.5, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[(5.0, Je suis d'accord avec tous les autres c...","[(1.0, My optometrist went in person to bring ...","[(4.5, Je suis arrivé à l'urgence de l'hôpital...",16,38.1,[Dr. Olivier appears to be an exceptionally sk...
Dr. Jean-Daniel Arbour,4.7,Ophthalmologist,16,[Dr Arbour est un excellent docteur. Compétent...,"[5.0, 5.0, 5.0, 2.0, 4.5, 5.0, 5.0, 5.0, 5.0, ...","[(5.0, Dr Arbour est un excellent docteur. Com...","[(2.0, Je n'ai jamais eu autant l'impression d...","[(5.0, I have a rare eye disease polypoidal ch...",3,18.8,[Dr. Arbour is highly regarded by his patients...
Dr. David Lederer,4.6,Ophthalmologist,49,[I am very thankful for having been able to se...,"[5.0, 1.0, 5.0, 5.0, 5.0, 3.5, 5.0, 5.0, 5.0, ...","[(5.0, I am very thankful for having been able...","[(1.0, Very very bad experience. He rush all t...","[(5.0, Dr. Lederer has now operated on both my...",9,18.4,[Dr. David Lederer is widely regarded as an ex...
Dr. Paul Thompson,4.6,Ophthalmologist,19,[Un Dr passionné par sa profession et attentio...,"[5.0, 5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 4.5, ...","[(5.0, Un Dr passionné par sa profession et at...","[(1.5, Le docteur Thomson ma opere pour une ca...","[(5.0, Après l'extraction de la cataracte de m...",1,5.3,[Dr. Paul Thompson generally receives high pra...
